In [1]:
from PIL import Image, ImageDraw
from path import Path
import numpy as np
from scipy import fftpack
import imageio

In [2]:
%ls

README.md            pilot_and_plane.png  plane_and_pilot.png
cv_lab2.ipynb        pilot_fft.png        plane_fft.png
cv_lab2.py           plane1.jpg
pilot.jpg            plane2.png


In [3]:
def display_np(x):
    """
    Display a numpy array as an image
    """
    result = Image.fromarray(x.astype('uint8'))
    result.show()

In [4]:
def read_image(x):
    x = Image.open(x).convert("L")
    x = np.array(x)
    return x

In [5]:
plane = read_image('plane2.png')
pilot = read_image("pilot.jpg")

In [6]:
display_np(pilot)
display_np(plane)

In [7]:
plane.shape, pilot.shape

((946, 1178), (470, 299))

In [8]:
r,c = 100,425
plane = plane[r:r + 470, c: c + 299]

In [9]:
plane.shape, pilot.shape

((470, 299), (470, 299))

In [10]:
# display_np(plane[0:300, 0:299])
# display_np(pilot[0:300, 0:299])

In [11]:
assert plane.shape == pilot.shape

In [12]:
pilot_fft = fftpack.fftshift(fftpack.fft2(pilot))
plane_fft = fftpack.fftshift(fftpack.fft2(plane))

In [13]:
imageio.imsave('pilot_fft.png', (np.log(abs(pilot_fft))* 255 /np.amax(np.log(abs(pilot_fft)))).astype(np.uint8))
imageio.imsave('plane_fft.png', (np.log(abs(plane_fft))* 255 /np.amax(np.log(abs(plane_fft)))).astype(np.uint8))

In [14]:
def filter_im(im, high = False, radius = 70):
    eX, eY = radius, radius
    x, y = im.shape[1], im.shape[0]
    bbox = (x/2 - eX/2, y/2 - eY/2, x/2 + eX/2, y/2 + eY/2)
    
    color = 255 if not high else 0
    fill  = 0 if not high else 255
    fpass = Image.new("L", (im.shape[1], im.shape[0]), color=color) 
    draw = ImageDraw.Draw(fpass)
    draw.ellipse(bbox, fill=fill)
    return np.array(fpass)

In [15]:
pilot_im = read_image('pilot_fft.png')
plane_im = read_image('plane_fft.png')

In [16]:
pilot_low = filter_im(pilot_im)
pilot_high = filter_im(pilot_im, True)

plane_low = filter_im(plane_im)
plane_high = filter_im(plane_im, True)

In [17]:
# display_np(pilot_low)
# display_np(pilot_high)

In [18]:
def join_im(f1, f2, im1, im2):
    assert im1.shape == im2.shape
    
    canvas = np.full(im1.shape, 255, dtype = complex)
    for i in range(im1.shape[0]):
        for j in range(im1.shape[1]):
            if f1[i,j] == 0: canvas[i,j] = im1[i,j]
            else: canvas[i,j] = im2[i,j]
    return canvas

In [19]:
pp1 = join_im(pilot_low, plane_high, pilot_fft, plane_fft)
pp2 = join_im(pilot_high, plane_low, pilot_fft, plane_fft)

In [20]:
ifft1 = abs(fftpack.ifft2(fftpack.ifftshift(pp1)))
ifft2 = abs(fftpack.ifft2(fftpack.ifftshift(pp2)))

In [21]:
display_np(ifft1)
display_np(ifft2)

In [22]:
imageio.imsave('plane_and_pilot.png', ifft1.astype(np.uint8))
imageio.imsave('pilot_and_plane.png', ifft2.astype(np.uint8))